In [42]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import json
from datetime import datetime,timezone

from dotenv import dotenv_values

from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest
from google.oauth2 import service_account

from dotenv import dotenv_values
# https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries
# https://codelabs.developers.google.com/codelabs/cloud-bigquery-python#0

# https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# https://cloud.google.com/python/docs/setup
# https://cloud.google.com/apis/docs/getting-started#enabling_apis

# https://cloud.google.com/bigquery/docs/reference/libraries
# https://cloud.google.com/python/docs/reference/bigquery/latest
# https://gcloud.readthedocs.io/en/latest/bigquery-client.html


In [43]:
# credentials = service_account.Credentials.from_service_account_file(r'C:\Windows\xxxxxx.json')
# print(credentials.project_id)

In [44]:
start_date_query='2020-01-01'

projectId='pongthorn'  # smart-data-ml
dataset_id='SMartDW'
table_id = f"{projectId}.{dataset_id}.incident"

env_path='.env'
config = dotenv_values(dotenv_path=env_path)

In [45]:
# client = bigquery.Client(credentials= credentials,project=projectId)
client = bigquery.Client(project=projectId)

In [46]:
try:
    table=client.get_table(table_id)  # Make an API request.
    print("Table {} already exists.".format(table_id))
    print(table.schema)
except NotFound:
    raise Exception("Table {} is not found.".format(table_id))

Table pongthorn.SMartDW.incident already exists.
[SchemaField('id', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('incident_no', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('severity_id', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('severity_name', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('sla', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('product_type', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('brand', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('model', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('incident_type', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('status', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('service_type', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('is_failure_type', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('count_detail', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('open_datetime', 'DAT

In [47]:
dt_imported=datetime.now(timezone.utc) # utc
#dt_imported=datetime.now() # utc
str_imported=dt_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Imported DateTime: {str_imported}" )

Imported DateTime: 2023-12-09 09:09:29


In [48]:
sql_lastImport=f"SELECT max(imported_at) as last_imported from `{table_id}` where open_datetime>='{start_date_query}' "
job_lastImported=client.query(sql_lastImport)
str_lastImported=None
for row in job_lastImported:    
    if row.last_imported is not None: 
        str_lastImported=row.last_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Last Imported DateTime: {str_lastImported}" )

if str_lastImported is not None:
  start_date_query=str_lastImported

print(f"Start Import on update_at of last imported date : {start_date_query}" )

Last Imported DateTime: None
Start Import on update_at of last imported date : 2020-01-01


In [49]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

In [52]:
# check schema 
#,timezone('UTC', incident.updated_at) as updated_at_utc
# and incident.updated_at>=%(start_date_param)s
sql_incident=f"""

select
incident.id as id, incident.incident_no as incident_no,

severity.id as  severity_id,
severity.severity_name as  severity_name,

service_level.sla_name as sla,

product_type.productype_name as product_type,brand.brand_name as brand,

xtype.incident_type_name as incident_type,
status.incident_status_name as status,
service.service_type_name service_type,

model.model_name as model,CASE WHEN failure_type IS NULL THEN  0 ELSE 1 END AS is_failure_type,


(select count(*) from  app_incident_detail  as detail where  detail.incident_master_id=incident.id ) as count_detail


,TO_CHAR(incident.incident_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as open_datetime
,TO_CHAR(incident.incident_close_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as close_datetime

,TO_CHAR(incident.incident_problem_start  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as response_datetime
,TO_CHAR(incident.incident_problem_end  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as resolved_datetime

,company.company_name as company
,TO_CHAR(incident.updated_at,'YYYY-MM-DD HH24:MI:SS') as updated_at 

from app_incident as incident
inner join app_incident_type as  xtype on incident.incident_type_id = xtype.id
inner join  app_incident_status as status on incident.incident_status_id = status.id
inner join  app_incident_severity as severity on  incident.incident_severity_id = severity.id
inner join  app_service_type as service on incident.service_type_id= service.id

inner join app_inventory as inventory on incident.inventory_id = inventory.id

inner join app_brand as brand on inventory.brand_id = brand.id
inner join app_model as model on inventory.model_id = model.id
inner join app_product_type as product_type on inventory.product_type_id = product_type.id
inner join app_sla as service_level on inventory.customer_sla_id = service_level.id

inner join app_project as project on inventory.project_id = project.id
inner join app_company as company on project.company_id = company.id

where incident.incident_status_id =4
and timezone('UTC', incident.updated_at)>='{start_date_query}'

order by incident.updated_at 

"""

# and timezone('UTC', incident.updated_at)>=%(start_date_param)s
print(sql_incident)



select
incident.id as id, incident.incident_no as incident_no,

severity.id as  severity_id,
severity.severity_name as  severity_name,

service_level.sla_name as sla,

product_type.productype_name as product_type,brand.brand_name as brand,

xtype.incident_type_name as incident_type,
status.incident_status_name as status,
service.service_type_name service_type,

model.model_name as model,CASE WHEN failure_type IS NULL THEN  0 ELSE 1 END AS is_failure_type,


(select count(*) from  app_incident_detail  as detail where  detail.incident_master_id=incident.id ) as count_detail


,TO_CHAR(incident.incident_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as open_datetime
,TO_CHAR(incident.incident_close_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as close_datetime

,TO_CHAR(incident.incident_problem_start  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as response_datetime
,TO_CHAR(incident.incident_problem_end  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI'

In [53]:
print("Create all issues dataframe")
#and timezone('UTC', incident.updated_at)>=%(start_date_param)s
# dict_params={"start_date_param":start_date_query}
# df_all=list_data(sql_incident,dict_params,get_postgres_conn())


df_all=list_data(sql_incident,dict_params,get_postgres_conn())

if df_all.empty==True:
    print("no transsaction update")
    exit()
else:
    print("import data to bigquery")
print(df_all.info())
df_all.tail()

Create all issues dataframe
import data to bigquery
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3967 entries, 0 to 3966
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 3967 non-null   int64 
 1   incident_no        3967 non-null   object
 2   severity_id        3967 non-null   int64 
 3   severity_name      3967 non-null   object
 4   sla                3967 non-null   object
 5   product_type       3967 non-null   object
 6   brand              3967 non-null   object
 7   incident_type      3967 non-null   object
 8   status             3967 non-null   object
 9   service_type       3967 non-null   object
 10  model              3967 non-null   object
 11  is_failure_type    3967 non-null   int64 
 12  count_detail       3967 non-null   int64 
 13  open_datetime      3967 non-null   object
 14  close_datetime     3967 non-null   object
 15  response_datetime  3967 non-null   ob

,id,incident_no,severity_id,severity_name,sla,product_type,brand,incident_type,status,service_type,model,is_failure_type,count_detail,open_datetime,close_datetime,response_datetime,resolved_datetime,company,updated_at
3962,4123,SR-ES-23-4123,3,Minor,24x7 6Hrs Resolution Time,Software,SAPB1,Software,Closed,Incident,SAPB1 Accounting and Finance Application,0,1,2023-11-28 11:09,2023-11-30 16:30,2023-11-28 11:09,2023-11-30 16:30,SRTET,2023-12-08 11:11:26
3963,4163,SR-ES-23-4163,2,Major,24x7 4Hrs Response Time,Firewall,Palo Alto,Software,Closed,Incident,PA-220,0,3,2023-12-06 16:18,2023-12-07 18:03,2023-12-06 19:17,2023-12-07 18:03,Coforge (NIIT),2023-12-08 11:21:44
3964,3993,SR-ES-23-3993,3,Minor,24x7 6Hrs Resolution Time,Software,CIMCO-CMMS,Software,Closed,Request,CIMCO-CMMS Maintenance Application,0,1,2023-11-13 15:00,2023-12-06 16:30,2023-11-13 15:01,2023-12-06 14:58,SRTET,2023-12-08 11:25:03
3965,4169,SR-ES-23-4169,2,Major,24x7 6Hrs Resolution Time,Switch,HPE,Configuration Change,Closed,Incident,Aruba 7010 (RW) 32 AP Branch Controller,0,1,2023-12-06 10:50,2023-12-08 16:30,2023-12-06 10:59,2023-12-08 16:30,SRTET,2023-12-08 15:47:56
3966,4109,SR-ES-23-4109,3,Minor,24x7 6Hrs Resolution Time,Server,HPE,Configuration Change,Closed,Incident,DL380 Gen10,0,1,2023-11-24 21:05,2023-12-07 16:30,2023-11-24 21:15,2023-12-07 16:30,SRTET,2023-12-08 15:50:24


In [54]:
# convert object to datetime
dateTimeCols=['open_datetime','response_datetime','resolved_datetime','close_datetime']
for col in dateTimeCols:
 df_all[col]=pd.to_datetime(df_all[col], format='%Y-%m-%d %H:%M',errors= 'coerce')

dateTimeCols2=['updated_at']
for col in dateTimeCols2:
 df_all[col]=pd.to_datetime(df_all[col], format='%Y-%m-%d %H:%M:%S',errors= 'coerce')

# UTC Import
df_all['imported_at']=dt_imported

In [55]:
df_all.dropna(inplace=True)

#df_all=df_all.head(10)
print(df_all.info())
df_all.head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 3966 entries, 0 to 3966
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   id                 3966 non-null   int64              
 1   incident_no        3966 non-null   object             
 2   severity_id        3966 non-null   int64              
 3   severity_name      3966 non-null   object             
 4   sla                3966 non-null   object             
 5   product_type       3966 non-null   object             
 6   brand              3966 non-null   object             
 7   incident_type      3966 non-null   object             
 8   status             3966 non-null   object             
 9   service_type       3966 non-null   object             
 10  model              3966 non-null   object             
 11  is_failure_type    3966 non-null   int64              
 12  count_detail       3966 non-null   int64             

,id,incident_no,severity_id,severity_name,sla,product_type,brand,incident_type,status,service_type,model,is_failure_type,count_detail,open_datetime,close_datetime,response_datetime,resolved_datetime,company,updated_at,imported_at
0,24,SR-ES-22-0024,1,Critical,24x7 4Hrs Resolution Time,Server,HPE,Hard Disk Drive Failure,Closed,Incident,SimpliVity 380 Gen9 Node,0,1,2022-03-24 17:14:00,2022-03-24 22:00:00,2022-03-24 17:18:00,2022-03-24 22:00:00,KTBCS,2022-03-24 22:08:41,2023-12-09 09:09:29.628598+00:00
1,8,SR-ES-22-0008,3,Minor,24x7 4Hrs Resolution Time,Storage,HPE,General Incident,Closed,Request,3PAR StoreServ 8400,0,1,2022-03-16 18:41:00,2022-03-17 10:00:00,2022-03-16 19:00:00,2022-03-17 10:00:00,KTBCS,2022-03-24 22:09:35,2023-12-09 09:09:29.628598+00:00
2,6,SR-ES-22-0006,3,Minor,24x7 4Hrs Resolution Time,Storage,HPE,General Incident,Closed,Request,3PAR Storageserv 8400 2N,0,1,2022-03-15 10:27:00,2022-03-15 12:00:00,2022-03-15 10:51:00,2022-03-15 12:00:00,KTBCS,2022-03-24 22:10:50,2023-12-09 09:09:29.628598+00:00
3,7,SR-ES-22-0007,1,Critical,9x5 NBD 4Hrs Response Time,Server,HPE,Memory Failure,Closed,Incident,DL385 Gen10+ 8SFF CTO Svr,0,1,2022-03-15 11:45:00,2022-03-15 17:30:00,2022-03-15 12:30:00,2022-03-15 17:30:00,KTBCS,2022-03-24 22:11:55,2023-12-09 09:09:29.628598+00:00
4,10,SR-ES-22-0010,1,Critical,24x7 4Hrs Resolution Time,Server,HPE,Hard Disk Drive Failure,Closed,Incident,SimpliVity 380 Gen9 Node,0,1,2022-03-17 14:40:00,2022-03-17 22:00:00,2022-03-17 15:00:00,2022-03-17 22:00:00,KTBCS,2022-03-24 22:12:15,2023-12-09 09:09:29.628598+00:00
5,11,SR-ES-22-0011,1,Critical,24x7 4Hrs Resolution Time,Server,HPE,Hard Disk Drive Failure,Closed,Incident,SimpliVity 380 Gen9 Node,0,1,2022-03-17 14:40:00,2022-03-17 22:00:00,2022-03-17 15:00:00,2022-03-17 22:00:00,KTBCS,2022-03-24 22:12:38,2023-12-09 09:09:29.628598+00:00
6,13,SR-ES-22-0013,1,Critical,24x7 4Hrs Resolution Time,Server,HPE,Hard Disk Drive Failure,Closed,Incident,SimpliVity 380 Gen9 Node,0,1,2022-03-21 10:10:00,2022-03-21 22:00:00,2022-03-21 10:30:00,2022-03-21 22:00:00,KTBCS,2022-03-24 22:13:02,2023-12-09 09:09:29.628598+00:00
7,9,SR-ES-22-0009,1,Critical,24x7 4Hrs Resolution Time,Server,HPE,Hard Disk Drive Failure,Closed,Incident,SimpliVity 380 Gen9 Node,0,1,2022-03-17 14:40:00,2022-03-17 22:00:00,2022-03-17 15:00:00,2022-03-17 22:00:00,KTBCS,2022-03-24 22:13:43,2023-12-09 09:09:29.628598+00:00
8,26,SR-ES-22-0026,1,Critical,24x7 4Hrs Resolution Time,Storage,NetApp,Hard Disk Drive Failure,Closed,Incident,FAS8020,0,1,2022-03-26 17:45:00,2022-03-27 22:00:00,2022-03-26 17:50:00,2022-03-27 22:00:00,KTBCS,2022-03-28 11:42:18,2023-12-09 09:09:29.628598+00:00
9,31,SR-ES-22-0031,2,Major,24x7 6Hrs Resolution Time,Storage,NetApp,General Incident,Closed,Incident,FAS8020,0,1,2022-03-20 23:16:00,2022-03-22 11:24:00,2022-03-20 23:45:00,2022-03-22 11:24:00,AIS,2022-03-28 21:34:34,2023-12-09 09:09:29.628598+00:00


In [56]:
def insertDataFrameToBQ(df_trasns):
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND",
        )

        job = client.load_table_from_dataframe(
            df_trasns, table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total ", len(df_trasns), "Imported closed incident to bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        print(e) 

try:
    insertDataFrameToBQ(df_all)
except Exception as ex:
    raise ex




Total  3966 Imported closed incident to bigquery successfully


In [57]:
importDate=dt_imported.strftime("%Y-%m-%d")

def load_data_bq(sql:str):
 query_result=client.query(sql)
 dfx=query_result.to_dataframe()
 return dfx
sql_query=f""" SELECT * FROM `{table_id}` WHERE imported_at >= '{importDate}' LIMIT 10 """
df=load_data_bq(sql_query)
df.head()

,id,incident_no,severity_id,severity_name,sla,product_type,brand,model,incident_type,status,service_type,is_failure_type,count_detail,open_datetime,close_datetime,response_datetime,resolved_datetime,company,updated_at,imported_at
0,1520,SR-ES-22-1520,1,Critical,24x7 6Hrs Resolution Time,Storage,YIP,dummy,Controller/Node Failure,Closed,Incident,1,1,2022-09-27 12:32:00,2022-09-28 11:43:00,2022-09-27 12:40:00,2022-09-27 14:00:00,PG,2022-09-28 11:48:54,2023-12-09 09:09:29.628598
1,619,SR-ES-22-0619,1,Critical,24x7 4Hrs Resolution Time,Storage,NetApp,FAS3240,Controller/Node Failure,Closed,Incident,0,2,2022-05-06 21:30:00,2022-05-07 05:00:00,2022-05-06 21:30:00,2022-05-07 05:00:00,MEA (ฝมธ.),2022-05-09 14:31:31,2023-12-09 09:09:29.628598
2,617,SR-ES-22-0617,1,Critical,24x7 4Hrs Resolution Time,Storage,NetApp,FAS3240,General Incident,Closed,Incident,0,2,2022-05-06 18:30:00,2022-05-07 05:00:00,2022-05-06 21:30:00,2022-05-07 05:00:00,MEA (ฝมธ.),2022-05-09 12:30:59,2023-12-09 09:09:29.628598
3,26,SR-ES-22-0026,1,Critical,24x7 4Hrs Resolution Time,Storage,NetApp,FAS8020,Hard Disk Drive Failure,Closed,Incident,0,1,2022-03-26 17:45:00,2022-03-27 22:00:00,2022-03-26 17:50:00,2022-03-27 22:00:00,KTBCS,2022-03-28 11:42:18,2023-12-09 09:09:29.628598
4,163,SR-ES-22-0163,1,Critical,24x7 4Hrs Resolution Time,Storage,NetApp,FAS8020,Hard Disk Drive Failure,Closed,Incident,0,0,2021-12-28 01:35:00,2021-12-28 10:22:00,2021-12-28 01:41:00,2021-12-28 10:22:00,KTBCS,2022-04-04 11:23:01,2023-12-09 09:09:29.628598
